In [1]:
import pandas as pd
import numpy as np
import time
import datetime
import os

In [21]:
# read all pkls in the folder into a list of dataframes
base_path = "/Users/finn/Documents/GitHub/FM-ds.github.io/EconOb/Prices/scrapes"

dfs = []

for path in os.listdir(base_path):
    if path.endswith('.pkl'):
        df = pd.read_pickle(f"{base_path}/{path}")
        dfs.append(df)
df = pd.concat(dfs)
df['tesco_id'] = df['link'].astype(str).replace("https://www.tesco.com/groceries/en-GB/products/", "", regex=True)

In [19]:
df['tesco_id'] = df['link'].astype(str).replace("https://www.tesco.com/groceries/en-GB/products/", "", regex=True)

In [23]:
df.to_csv("all_scrapes_03_07_2023.csv")

In [9]:
df['']

,name,link,prices,category,price,relative_price,scrape_time,source_page
0,Activia Rhubarb Yogurt 4X115g,https://www.tesco.com/groceries/en-GB/products...,"[£2.50, £0.54/100g]",fresh,2.50,£0.54/100g,1.687320e+09,NaN
1,Danone Actimel Multifruit Drink 12X100g,https://www.tesco.com/groceries/en-GB/products...,"[£4.50, £0.38/100g]",fresh,4.50,£0.38/100g,1.687320e+09,NaN
2,Tesco British Salted Block Butter 250G ++,https://www.tesco.com/groceries/en-GB/products...,"[£1.89, £7.56/kg]",fresh,1.89,£7.56/kg,1.687320e+09,NaN
3,Tesco British Unsalted Butter 250G,https://www.tesco.com/groceries/en-GB/products...,"[£1.89, £7.56/kg]",fresh,1.89,£7.56/kg,1.687320e+09,NaN
4,Tesco Semi Skimmed Milk 1.13L/2 Pints,https://www.tesco.com/groceries/en-GB/products...,"[£1.25, £1.11/litre]",fresh,1.25,£1.11/litre,1.687320e+09,NaN
...,...,...,...,...,...,...,...,...
27018,Purina One Wet Chicken Beef 8X85g,https://www.tesco.com/groceries/en-GB/products...,"[£4.60, £6.76/kg]",NaN,4.60,£6.76/kg,1.688064e+09,None
27019,Sheba Fine Flakes In Gravy Fish Selection 12 X...,https://www.tesco.com/groceries/en-GB/products...,"[£5.50, £5.39/kg]",NaN,5.50,£5.39/kg,1.688064e+09,None
27020,Pedigree Adult Cuts In Jelly Multi Pack 6 X 385G,https://www.tesco.com/groceries/en-GB/products...,"[£7.00, £3.03/kg]",NaN,7.00,£3.03/kg,1.688064e+09,None
27021,Lilys Kitchen Mini Beef Brgrs For Dogs 70G,https://www.tesco.com/groceries/en-GB/products...,"[£3.50, £50.00/kg]",NaN,3.50,£50.00/kg,1.688064e+09,None


In [4]:
dfs[5].to_csv("scrape_sample.csv")

In [8]:
dfs[5][['name', 'link', 'scrape_time', 'prices', 'price', 'relative_price']].to_csv("scrape_sample.csv")

In [29]:
counts =  df.groupby('link').aggregate({'price': 'count', 'link': 'first'})
df['date'] = pd.to_datetime(df['scrape_time'])
in_all_ids = counts[counts['price'] > 1]['link']
df = df[df['link'].isin(in_all_ids)]


In [30]:
# lose the time
df['date'].dt.date


1        1970-01-01
10       1970-01-01
11       1970-01-01
16       1970-01-01
19       1970-01-01
            ...    
26919    1970-01-01
26922    1970-01-01
26923    1970-01-01
26938    1970-01-01
27001    1970-01-01
Name: date, Length: 5845, dtype: object

In [17]:
pd.concat(dfs)

,name,link,prices,category,price,relative_price,scrape_time,source_page
0,Activia Rhubarb Yogurt 4X115g,https://www.tesco.com/groceries/en-GB/products...,"[£2.50, £0.54/100g]",fresh,2.50,£0.54/100g,1.687320e+09,NaN
1,Danone Actimel Multifruit Drink 12X100g,https://www.tesco.com/groceries/en-GB/products...,"[£4.50, £0.38/100g]",fresh,4.50,£0.38/100g,1.687320e+09,NaN
2,Tesco British Salted Block Butter 250G ++,https://www.tesco.com/groceries/en-GB/products...,"[£1.89, £7.56/kg]",fresh,1.89,£7.56/kg,1.687320e+09,NaN
3,Tesco British Unsalted Butter 250G,https://www.tesco.com/groceries/en-GB/products...,"[£1.89, £7.56/kg]",fresh,1.89,£7.56/kg,1.687320e+09,NaN
4,Tesco Semi Skimmed Milk 1.13L/2 Pints,https://www.tesco.com/groceries/en-GB/products...,"[£1.25, £1.11/litre]",fresh,1.25,£1.11/litre,1.687320e+09,NaN
...,...,...,...,...,...,...,...,...
27018,Purina One Wet Chicken Beef 8X85g,https://www.tesco.com/groceries/en-GB/products...,"[£4.60, £6.76/kg]",NaN,4.60,£6.76/kg,1.688064e+09,None
27019,Sheba Fine Flakes In Gravy Fish Selection 12 X...,https://www.tesco.com/groceries/en-GB/products...,"[£5.50, £5.39/kg]",NaN,5.50,£5.39/kg,1.688064e+09,None
27020,Pedigree Adult Cuts In Jelly Multi Pack 6 X 385G,https://www.tesco.com/groceries/en-GB/products...,"[£7.00, £3.03/kg]",NaN,7.00,£3.03/kg,1.688064e+09,None
27021,Lilys Kitchen Mini Beef Brgrs For Dogs 70G,https://www.tesco.com/groceries/en-GB/products...,"[£3.50, £50.00/kg]",NaN,3.50,£50.00/kg,1.688064e+09,None


In [4]:
all_files

['failure_log.txt',
 'scrapes',
 'products_starttime.json',
 'products_1687613700.4520671.json',
 'products_fresh_food.pkl',
 'batch.ipynb',
 '.DS_Store',
 'products_cupboard.pkl',
 'tesco_prices_with_embeddings.csv',
 'products_1687876257.791008.json',
 'products_1687608807.858767.json',
 'cpi_with_embeddings.pkl',
 'product_pages.pkl',
 'products_pets.pkl',
 'products_home_ents.pkl',
 'products_household.pkl',
 'products_bakery.pkl',
 'products_1687902012.0459142.json',
 'products_frozen.pkl',
 'geckodriver.log',
 'products_treats_snacks.pkl',
 'products_health_beauty.pkl',
 'products_baby_todler.pkl',
 'products_1687609000.537618.json',
 'products_1687611855.822231.json',
 'products_1687745724.376501.json',
 'products_drinks.pkl',
 'success_log.txt',
 '2023_06_24_tesco.pkl',
 'quants',
 'tesco_prices_with_embeddings.pkl',
 'Selenium_Tesco.ipynb']